In [434]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen, Request
from nltk.stem.snowball import SnowballStemmer
import string
from nltk.corpus import stopwords
import csv
import time
import io

stopwords = set(stopwords.words('spanish'))
political_keys = r"C:\Users\meiza\Documents\GitHub\Machine-Learning\Political Trends in FC - RNN\Dataset\political_keys.csv"
dataset = r"C:\Users\meiza\Documents\GitHub\Machine-Learning\Political Trends in FC - RNN\Dataset\dataset.csv"

In [424]:
#A PARTIR DE UN HILO DEVOLVEMOS TÍTULO Y MENSAJES

def Webscrap(hilo):
    #time.sleep(1)
    
    #REALIZAMOS LA LLAMADA
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
    root_url = "https://www.forocoches.com/foro/showthread.php?t="
    req = Request(url=root_url+hilo, headers=headers) 
    client = urlopen(req)
    page_html = client.read()
    client.close()
    
    #COGEMOS LOS MENSAJES Y TITULO
    page_soup = soup(page_html, "html5lib")
    titulo = page_soup.findAll("span", {"class": "cmega"})
    if not(titulo == []):
        titulo = titulo[0].text
        mensajes = page_soup.findAll("div", {"id": lambda x: x and x.startswith('post_message')})
    
        #QUITAMOS QUOTES
        i = 0
        for mensaje in mensajes:
            if (mensaje.div != None):
                mensaje.div.extract()
            #SÓLO EL TEXTO
            mensajes[i] = mensaje.text
        
            #QUITAMOS LOS PARRAFOS Y DOBLES ESPACIOS
            mensajes[i] = mensajes[i].replace('\n', ' ').replace('\r', '').lower()
            mensajes[i] = ' '.join(mensajes[i].split())
            i += 1

        return titulo,mensajes
    else:
        return None,None

In [425]:
#NORMALIZAMOS EL TEXTO
def Normalize(text):
    #QUITAMOS ACENTOS
        
    
    #QUITAMOS \n y \r. TEXTO EN MINÚSCULA
    text = text.replace('\n', ' ').replace('\r', '').lower()
    
    #QUITAMOS NÚMEROS
    text = ''.join([i for i in text if not i.isdigit()])
    
    #QUITAMOS PUNTUACIÓN
    text = "".join(l for l in text if l not in (string.punctuation,'?','¿',',','.'))
    
    #QUITAMOS DOBLES ESPACIOS
    text = ' '.join(text.split())

    #STEMMING Y QUITAMOS STOPWORDS
    stemmer = SnowballStemmer("spanish")
    words = text.split()
    text = ''
    for word in words:
        if not word in stopwords:
            stemmed_word = stemmer.stem(word)
            text = text + " " + stemmed_word
    
    return text

In [426]:
#FUNCIÓN PARA VER SI EL TÍTULO INDICA QUE ES UN HILO POLÍTICO

def Ispolitical(titulo):
    political = False
    
    #Normalizamos titulo
    titulo = Normalize(titulo)
    titulo = titulo.split()

    #Cruzamos nuestro título con nuestras palabras clave para saber si es título político
    with open(political_keys, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            key = Normalize(row[0]).strip()
            for palabra in titulo:
                if (key == palabra):
                    political = True
    return political    

In [476]:
#FUNCIÓN PARA ECSRIBIR EN EL DATASET

def Writedatasetpolitico(mensajes):
    with io.open(dataset, 'a', newline='',encoding="utf-8") as ds:
        dataset_writer = csv.writer(ds, delimiter=',', quotechar="'", quoting=csv.QUOTE_NONNUMERIC)
        for mensaje in mensajes:
            dataset_writer.writerow([1, mensaje])

def Writedatasetnopolitico(mensajes):
    with io.open(dataset, 'a', newline='',encoding="utf-8") as ds:
        dataset_writer = csv.writer(ds, delimiter=',', quotechar="'", quoting=csv.QUOTE_NONNUMERIC)
        for mensaje in mensajes:
            dataset_writer.writerow([0, mensaje])

In [420]:
#BUSCADOR DE HILOS POLITIZADOS PARA AÑADIRLOS AL DATASET

hilo_inicio=7991190
hilo_final=7991255

for i in range(hilo_inicio, hilo_final+1):
    titulo, mensajes = Webscrap(str(i))
    if titulo != None:
        politico = Ispolitical(titulo)
        if politico:
            print(titulo)
            Writedatasetpolitico(mensajes)

In [507]:
#AÑADIR HILOS AL DATASET MANUALMENTE

hilo='7991527'
titulo, mensajes = Webscrap(hilo)
Writedatasetnopolitico(mensajes)